In [1]:
import pandas as pd
import glob

headers = ['Date','Time','Open','High','Low','Close','Volume']
path = "C:\\Users\\desma\\Downloads\\jupyter work\\DATA\\"
#set up so i could run multiple datasets if wanted
filenames = glob.glob(path + "SPY2h.txt")
for file in filenames:
    print("\nReading file = ", file)
    df = pd.read_csv(file, header = None)
    df.columns=headers


Reading file =  C:\Users\desma\Downloads\jupyter work\DATA\SPY2h.txt


In [7]:
import pandas_ta as ta

fast = 90
slow = 200
df['FastSMA'] = ta.sma(df['Close'], fast)
df['SlowSMA'] = ta.sma(df['Close'], slow)

#reset index so we are not missing any Moving Average values. (*makes signal conditions easier*)
df.dropna(inplace=True)
df.reset_index(inplace=True)
dfLen = len(df)
dfLen

8746

In [17]:
signal = [0]*dfLen
for row in range(1, dfLen):
    prevFast = df.FastSMA[row-1]
    currFast = df.FastSMA[row]
    slowMA = df.SlowSMA[row]
    
    #long position --> signal == 1
    if prevFast<slowMA and currFast>slowMA:
        signal[row] = 1
        
    #short position --> signal == 2
    elif prevFast>slowMA and currFast<slowMA:
        signal[row] = 2
        
df['Signal'] = signal
df['Signal'].value_counts()

0    8709
2      21
1      16
Name: Signal, dtype: int64

In [16]:
import plotly.graph_objects as go

def plotSignal(df):
    fig = go.Figure(data=[go.Candlestick(x=df.index, open=df['Open'],  high=df['High'], low=df['Low'],
                                         close=df['Close'], name='SPY 2h')])
    
    fig.add_scatter(x=df.index, y=df['FastSMA'], mode="markers",
                marker=dict(size=1, color="Black"),
                name="Fast Moving Average")
    
    fig.add_scatter(x=df.index, y=df['SlowSMA'], mode="markers",
                marker=dict(size=2, color="Blue"),
                name="Slow Moving Average")
    fig.show()
    
df1 = df[0:1200]
plotSignal(df1)

In [19]:
def SIGNAL():
    return df.Signal

In [25]:
from backtesting import Strategy, Backtest
import numpy as np

class myStrat(Strategy):
    def init(self):
        super().init()
        self.signal1=self.I(SIGNAL)     
        
    def next(self):
        super().next()
        trailSL = self.data.Close[-1]*.01
        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - trailSL)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + trailSL)
        
        if self.signal1==1 and len(self.trades)==0: #Long Signal
            self.buy()
            
        if self.signal1==2 and len(self.trades)==0: #Short Signal
            self.sell()
            
bt = Backtest(df, myStrat, cash=100_000, margin = 1/1, commission=.000)
stat = bt.run()
stat
bt.plot(show_legend=False, superimpose = True)

C:\Users\desma\AppData\Local\Temp/ipykernel_14748/1494666111.py:24: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Row(id='3028', ...)